In [1]:
import numpy as np
import pandas as pd

In [21]:
def calculate_total_return(coupon_rate, bond_prices):
    """ 債券トータルリターンを計算する関数
    
    Args :
        coupon_rate : float
            クーポンレート (時点に依らず不変)
        bond_price : 

    """
    
    #　債券の額面 
    face_value = 100
    
    # インカムゲイン
    coupon_income = (coupon_rate / 100) * face_value
    # キャピタルゲイン
    capital_gain = np.diff(bond_prices)
    
    # トータルリターン
    total_return = (coupon_income + capital_gain) / bond_prices[0] # 初期時点の価格で割る
    
    return total_return


In [22]:
import pandas as pd

# 例としてのデータフレーム
df = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'C', 'B', 'C', 'A'],
    'Value': [10, 20, 15, 30, 25, 35, 40]
})

# 'Category' カラムに基づいてグループ化
grouped = df.groupby('Category')

# 各グループに対して何か操作を行う
for name, group in grouped:
    print(f"Category: {name}")
    print(group)

Category: A
  Category  Value
0        A     10
2        A     15
6        A     40
Category: B
  Category  Value
1        B     20
4        B     25
Category: C
  Category  Value
3        C     30
5        C     35


In [56]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

# 50個のユニークなISINコードを生成
unique_isins = ['ISIN' + str(i).zfill(3) for i in range(1, 51)]

# 信用格付けのリスト
credit_ratings = ['AAA', 'BBB', 'CCC', 'DDD']

# ベースとなる日付をdatetimeオブジェクトに変換
base_date = datetime.strptime('2023-01-01', '%Y-%m-%d')

# 100時点分のデータセットを生成
num_points = 100
data_list = []

# 各ISINコードに対してデータを生成
for isin in unique_isins:
    for day_offset in range(num_points):
        current_date = base_date + timedelta(days=day_offset)
        bond_price = 100 + np.random.randint(-2, 3)  # ボンドの価格をランダムに変動させる
        coupon_rate = 5  # 一定のクーポンレート
        credit_rating = np.random.choice(credit_ratings)
        data_list.append({
            'Date': current_date,
            'ISIN': isin,
            'Bond_Price': bond_price,
            'Coupon_Rate': coupon_rate,
            'Credit_Rating': credit_rating
        })

# 生成されたデータフレームを結合
df = pd.DataFrame(data_list)

# 最初のいくつかのデータを表示
df


,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating
0,2023-01-01,ISIN001,102,5,AAA
1,2023-01-02,ISIN001,101,5,DDD
2,2023-01-03,ISIN001,101,5,BBB
3,2023-01-04,ISIN001,102,5,CCC
4,2023-01-05,ISIN001,101,5,DDD
...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC
4996,2023-04-07,ISIN050,98,5,DDD
4997,2023-04-08,ISIN050,102,5,BBB
4998,2023-04-09,ISIN050,101,5,CCC


In [57]:
import pandas as pd

def calculate_total_return(df):
    # 前時点との債券価格の差分を計算
    df['Price_Diff'] = df['Bond_Price'].diff().fillna(0)
    # インカムゲインの計算
    df['Income_Gain'] = (df['Coupon_Rate'] / 100) * 100  # 額面価格100と仮定
    # トータルリターンの計算
    df['Total_Return'] = ((df['Price_Diff'] + df['Income_Gain']) / df['Bond_Price'].shift())
    # 
    df['Total_Return'].fillna(0, inplace=True)
    
    return df

# ISINコードごとにグループ化し、各グループに対してトータルリターンを計算
df = df.groupby('ISIN').apply(calculate_total_return).reset_index(drop = True)

df.to_csv("test.csv")

In [58]:
# 累積リターンを計算する関数
def calculate_cumulative_return(df):
    df['Cumulative_Return'] = df.groupby('ISIN')['Total_Return'].cumsum()
    return df

# 累積リターンを計算
df = calculate_cumulative_return(df).reset_index(drop = True)

df

,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating,Price_Diff,Income_Gain,Total_Return,Cumulative_Return
0,2023-01-01,ISIN001,102,5,AAA,0.0,5.0,0.000000,0.000000
1,2023-01-02,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.039216
2,2023-01-03,ISIN001,101,5,BBB,0.0,5.0,0.049505,0.088721
3,2023-01-04,ISIN001,102,5,CCC,1.0,5.0,0.059406,0.148127
4,2023-01-05,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.187342
...,...,...,...,...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC,0.0,5.0,0.051020,4.745981
4996,2023-04-07,ISIN050,98,5,DDD,0.0,5.0,0.051020,4.797001
4997,2023-04-08,ISIN050,102,5,BBB,4.0,5.0,0.091837,4.888838
4998,2023-04-09,ISIN050,101,5,CCC,-1.0,5.0,0.039216,4.928054


In [61]:
# 判定に使用するカラム名のリスト
columns_to_check = ['A', 'B', 'C']

def check_green_flag(columns_to_check):
    # 新しいカラム 'merged_green_flag' を追加
    df['merged_green_flag'] = df[columns_to_check].max(axis=1)

df

,Date,ISIN,Bond_Price,Coupon_Rate,Credit_Rating,Price_Diff,Income_Gain,Total_Return,Cumulative_Return
0,2023-01-01,ISIN001,102,5,AAA,0.0,5.0,0.000000,0.000000
1,2023-01-02,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.039216
2,2023-01-03,ISIN001,101,5,BBB,0.0,5.0,0.049505,0.088721
3,2023-01-04,ISIN001,102,5,CCC,1.0,5.0,0.059406,0.148127
4,2023-01-05,ISIN001,101,5,DDD,-1.0,5.0,0.039216,0.187342
...,...,...,...,...,...,...,...,...,...
4995,2023-04-06,ISIN050,98,5,CCC,0.0,5.0,0.051020,4.745981
4996,2023-04-07,ISIN050,98,5,DDD,0.0,5.0,0.051020,4.797001
4997,2023-04-08,ISIN050,102,5,BBB,4.0,5.0,0.091837,4.888838
4998,2023-04-09,ISIN050,101,5,CCC,-1.0,5.0,0.039216,4.928054


In [64]:
# 信用格付けと日付ごとにグループ化し、平均値を計算
average_return = df.groupby(['Credit_Rating', 'Date'])["Cumulative_Return"].mean().reset_index()

In [65]:
average_return

,Credit_Rating,Date,Cumulative_Return
0,AAA,2023-01-01,0.000000
1,AAA,2023-01-02,0.051660
2,AAA,2023-01-03,0.109695
3,AAA,2023-01-04,0.158381
4,AAA,2023-01-05,0.209132
...,...,...,...
395,DDD,2023-04-06,4.769021
396,DDD,2023-04-07,4.819908
397,DDD,2023-04-08,4.873830
398,DDD,2023-04-09,4.924756
